<a href="https://colab.research.google.com/github/Boonyaratt/LGBM_recommendation/blob/master/Notebooks/onlyfeatures_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import warnings
import lightgbm as lgb
import pathlib
import zipfile
import pandas as pd

warnings.filterwarnings('ignore')
np.random.seed(42)

In [2]:
from google.colab import files
uploaded = files.upload()

Saving features_all3.zip to features_all3 (1).zip


In [3]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [4]:
# Define paths
zip_file_path = '/content/features_all3.zip' # Adjust path to your zip file
extract_path = '/content/dataset' # Desired extraction directory in Colab

# Create a ZipFile object and extract
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [35]:
features_all = pd.read_csv('/content/dataset/features_all3.csv')
features_all.head()

,household_key,BASKET_ID,PROMO_KEY,day_t,qid,label,feat_min_recency,feat_mean_recency,feat_seen_products,overlap_decay_h3,...,feat_overlap_ratio,feat_jaccard,AGE_19_24,AGE_25_34,AGE_35_44,AGE_45_54,AGE_55_64,AGE_65P,AGE_UNK,age_mid
0,1,31172831466,54850010009|29,282,1_31172831466,0,8,34.0,4,0.182293,...,0.013889,0.011173,0,0,0,0,0,1,0,70
1,1,31172831466,57800000033|29,282,1_31172831466,0,9999,9999.0,0,0.000000,...,0.000000,0.000000,0,0,0,0,0,1,0,70
2,1,31172831466,54300021057|29,282,1_31172831466,0,9999,9999.0,0,0.000000,...,0.000000,0.000000,0,0,0,0,0,1,0,70
3,1,31172831466,51380041013|29,282,1_31172831466,0,9999,9999.0,0,0.000000,...,0.000000,0.000000,0,0,0,0,0,1,0,70
4,1,31172831466,54300016033|29,282,1_31172831466,0,9999,9999.0,0,0.000000,...,0.000000,0.000000,0,0,0,0,0,1,0,70


##Clean

In [36]:
df = features_all.copy()

In [37]:
def make_time_series_cv_splits(base, n_splits=5, purge_days=7):
    days = np.sort(base['day_t'].unique())
    cut_idx = np.linspace(0, len(days), n_splits+1, dtype=int)
    folds = []
    for i in range(n_splits):
        va_start = days[cut_idx[i]]
        va_end   = days[cut_idx[i+1]-1] if cut_idx[i+1]-1 < len(days) else days[-1]
        tr_end   = va_start - purge_days
        train_mask_f = base['day_t'] <= tr_end
        valid_mask_f = (base['day_t'] >= va_start) & (base['day_t'] <= va_end)
        if valid_mask_f.sum() == 0 or train_mask_f.sum() == 0:
            continue
        grp_tr_f = base.loc[train_mask_f].groupby('qid', sort=False).size().tolist()
        grp_va_f = base.loc[valid_mask_f].groupby('qid', sort=False).size().tolist()
        folds.append((train_mask_f, valid_mask_f, grp_tr_f, grp_va_f))
    return folds

def make_time_based_split(base, test_ratio=0.2, purge_days=7):
    days = np.sort(base['day_t'].unique())
    cutoff = days[int(len(days)*(1 - test_ratio))]
    tr_end = cutoff - purge_days
    train_mask = base['day_t'] <= tr_end
    valid_mask = base['day_t'] >= cutoff
    grp_tr = base.loc[train_mask].groupby('qid').size().tolist()
    grp_va = base.loc[valid_mask].groupby('qid').size().tolist()
    print(f"Train {base.loc[train_mask,'day_t'].min()}–{base.loc[train_mask,'day_t'].max()} | "
          f"Valid {base.loc[valid_mask,'day_t'].min()}–{base.loc[valid_mask,'day_t'].max()} | "
          f"Purge={purge_days}d")
    return train_mask, valid_mask, grp_tr, grp_va

In [38]:
cv_folds = make_time_series_cv_splits(df, n_splits=5, purge_days=7)
len(cv_folds), [ (mtr.sum(), mva.sum()) for mtr,mva,_,_ in cv_folds ]

(4,
 [(np.int64(44160), np.int64(84130)),
  (np.int64(114571), np.int64(184340)),
  (np.int64(298155), np.int64(210993)),
  (np.int64(509705), np.int64(163496))])

In [39]:
train_mask, valid_mask, grp_tr, grp_va = make_time_based_split(df, test_ratio=0.2, purge_days=7)
max_train_day = int(df.loc[train_mask, 'day_t'].max())

Train 224–607 | Valid 614–711 | Purge=7d


In [40]:
# สร้างรายการคอลัมน์ผู้สมัครจากทุกกลุ่ม (numeric/bool เท่านั้น)
id_cols = [c for c in ['household_key','BASKET_ID','PROMO_KEY','day_t','qid','label'] if c in df.columns]
all_numeric = df.drop(columns=id_cols, errors='ignore').select_dtypes(include=['number','bool']).copy()
for c in all_numeric.columns:
    if all_numeric[c].dtype == bool:
        all_numeric[c] = all_numeric[c].astype(np.uint8)

# 1) ตัดคอลัมน์คงที่/เกือบคงที่
nunq = all_numeric.nunique(dropna=False)
cand = all_numeric.loc[:, nunq > 1].copy()

# 2) redundancy filter ด้วย Spearman corr (เก็บฟีเจอร์ตัวแทนแบบ greedy)
corr = cand.corr(method='spearman').abs()
cols = corr.columns.tolist()
selected_pref = []
blocked = set()
# จัดอันดับตาม variance สูงก่อน
var_order = cand.var().sort_values(ascending=False).index.tolist()
for c in var_order:
    if c in blocked:
        continue
    selected_pref.append(c)
    high_corr = corr.index[(corr[c] > 0.95)].tolist()
    blocked.update(high_corr)

print(f'Prefilter kept {len(selected_pref)} of {cand.shape[1]} columns')

Prefilter kept 48 of 64 columns


In [41]:
# === Train-only Target Encoding helpers (no leakage) ===
def compute_te(train_df, group_cols, target_col='label', m=50, prior=None, out_name='te'):
    g = train_df.groupby(group_cols)[target_col].agg(mean='mean', count='count').reset_index()
    if prior is None:
        prior = float(train_df[target_col].mean())
    g[out_name] = (g['count']*g['mean'] + m*prior) / (g['count'] + m)
    return g[[*(group_cols if isinstance(group_cols, (list, tuple)) else [group_cols]), out_name]]

tr = df.loc[train_mask, ['household_key','PROMO_KEY','label']].copy()
global_mean = float(tr['label'].mean())

# PROMO prior CTR (train-only)
promo_te = compute_te(tr, 'PROMO_KEY', m=100, prior=global_mean, out_name='promo_te')
df = df.merge(promo_te, on='PROMO_KEY', how='left')
df['promo_te'] = df['promo_te'].fillna(global_mean).astype(np.float32)

# USER (household) prior
hh_te = compute_te(tr, 'household_key', m=200, prior=global_mean, out_name='hh_te')
df = df.merge(hh_te, on='household_key', how='left')
df['hh_te'] = df['hh_te'].fillna(global_mean).astype(np.float32)

# USER×PROMO affinity
hh_promo_te = compute_te(tr, ['household_key','PROMO_KEY'], m=200, prior=global_mean, out_name='hh_promo_te')
df = df.merge(hh_promo_te, on=['household_key','PROMO_KEY'], how='left')
df['hh_promo_te'] = df['hh_promo_te'].fillna(df['hh_te']).astype(np.float32)

if 'AGE_BAND' in df.columns:
    age_te = compute_te(tr.join(df[['AGE_BAND']], how='left'), 'AGE_BAND', m=100, prior=global_mean, out_name='age_te')
    df = df.merge(age_te, on='AGE_BAND', how='left')
    df['age_te'] = df['age_te'].fillna(global_mean).astype(np.float32)

    age_promo_te = compute_te(tr.join(df[['AGE_BAND']], how='left'), ['AGE_BAND','PROMO_KEY'], m=200, prior=global_mean, out_name='age_promo_te')
    df = df.merge(age_promo_te, on=['AGE_BAND','PROMO_KEY'], how='left')
    df['age_promo_te'] = df['age_promo_te'].fillna(df['age_te']).astype(np.float32)

In [42]:
def cv_feature_importance_ranker(df, features, folds, params=None, early_stopping=100):
    if params is None:
        params = dict(objective='lambdarank', metric='ndcg', boosting_type='gbdt',
                      num_leaves=47, learning_rate=0.06, n_estimators=600,
                      subsample=0.9, colsample_bytree=0.9, min_data_in_leaf=100,
                      lambda_l2=1.0, random_state=42, verbose=-1, lambdarank_truncation_level=10)
    feats = features
    imps = []
    for (tr_m, va_m, grp_tr_f, grp_va_f) in folds:
        X_tr = df.loc[tr_m, feats]; y_tr = df.loc[tr_m, 'label'].astype(int).values
        X_va = df.loc[va_m, feats]; y_va = df.loc[va_m, 'label'].astype(int).values
        ranker = lgb.LGBMRanker(**params)
        ranker.fit(X_tr, y_tr, group=grp_tr_f,
                   eval_set=[(X_va, y_va)], eval_group=[grp_va_f],
                   eval_at=[5,10], callbacks=[lgb.early_stopping(early_stopping, first_metric_only=True)])
        imp_gain = pd.Series(ranker.booster_.feature_importance(importance_type='gain'),
                             index=feats, dtype=float)
        imps.append(imp_gain)
    I = pd.concat(imps, axis=1)
    I.columns = [f'fold{i+1}' for i in range(I.shape[1])]
    I['gain_mean'] = I.mean(axis=1)
    I['gain_std']  = I.std(axis=1)
    I['gain_rank'] = I['gain_mean'].rank(ascending=False, method='average')
    return I.sort_values('gain_mean', ascending=False)

# ใช้คอลัมน์หลัง prefilter
imp_df = cv_feature_importance_ranker(df, selected_pref, cv_folds)
topN = 200  # ปรับได้ (เช่น 100/150/300)
top_cols_cv = imp_df.head(topN).index.tolist()
len(top_cols_cv), top_cols_cv[:10]

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	valid_0's ndcg@5: 0.987698	valid_0's ndcg@10: 0.98913
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[318]	valid_0's ndcg@5: 0.980387	valid_0's ndcg@10: 0.982315
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	valid_0's ndcg@5: 0.982713	valid_0's ndcg@10: 0.984705
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's ndcg@5: 0.986557	valid_0's ndcg@10: 0.988351
Evaluated only: ndcg@5


(48,
 ['promo_offer_ct',
  'feat_jaccard',
  'feat_num_promo_products',
  'feat_promo_pos_rate',
  'feat_min_recency',
  'feat_mean_recency',
  'feat_user_dept_aff',
  'feat_user_brand_aff',
  'feat_hist_size',
  'feat_seen_products'])

In [43]:
def pick(df, cols): return [c for c in cols if c in df.columns]

groups = {
    'similarity': pick(df, ['feat_hist_size','feat_overlap_cnt','feat_overlap_ratio','feat_jaccard']),
    'recency': pick(df, ['feat_min_recency','feat_mean_recency','feat_seen_products']),
    'recency_decay': [c for c in df.columns if c.startswith('overlap_decay_h')],
    'affinity': pick(df, ['feat_user_dept_aff','feat_user_brand_aff']),
    'immediate': pick(df, ['seen_in_1d','seen_in_3d','seen_in_7d','short_overlap_7d']),
    'coverage': pick(df, ['feat_num_promo_products','feat_num_promo_departments','feat_num_promo_brands']),
    'popularity': [c for c in df.columns if c.startswith('feat_promo_')] + pick(df, ['promo_offer_ct','promo_pos_ct','promo_unique_users']),
    'temporal': pick(df, ['feat_dayofweek','feat_is_weekend']),
    'promo_ohe': [c for c in df.columns if c.startswith('DEPARTMENT_') or c.startswith('BRAND_')],
}

feature_groups = {g: df[cols].select_dtypes(include=['number','bool']) for g, cols in groups.items() if len(cols) > 0}
for g, F in feature_groups.items():
    for c in F.columns:
        if F[c].dtype == bool:
            feature_groups[g][c] = F[c].astype(np.uint8)

In [44]:
# Add TE feature groups
te_cols = [c for c in ['promo_te','hh_te','hh_promo_te','age_te','age_promo_te'] if c in df.columns]
if te_cols:
    feature_groups['te'] = df[te_cols].astype(np.float32)

In [45]:
allowed = set(top_cols_cv)
for g in list(feature_groups.keys()):
    cols_g = [c for c in feature_groups[g].columns if c in allowed]
    if cols_g:
        feature_groups[g] = feature_groups[g][cols_g]
    else:
        feature_groups.pop(g, None)

In [46]:
def downsample_train_by_qid(df, train_mask, R=15, hardness_cols=('feat_promo_pos_rate',), rng_seed=42):
    rng = np.random.RandomState(rng_seed)
    tr = df.loc[train_mask].copy()
    if hardness_cols:
        cols_present = [c for c in hardness_cols if c in tr.columns]
        if cols_present:
            z = []
            for c in cols_present:
                s = pd.to_numeric(tr[c], errors='coerce').fillna(0.0)
                z.append(s.rank(pct=True).astype(float))
            tr['_hard'] = np.vstack(z).mean(axis=0)
        else:
            tr['_hard'] = 0.0
    else:
        tr['_hard'] = 0.0

    keep_idx = []
    for qid, g in tr.groupby('qid', sort=False):
        pos = g.index[g['label']==1].to_numpy()
        neg = g.index[g['label']==0].to_numpy()
        if len(pos)==0: continue
        n_keep = min(len(neg), R*len(pos))
        if n_keep>0 and g['_hard'].notnull().any():
            neg_keep = g.loc[neg].sort_values('_hard', ascending=False).index.to_numpy()[:n_keep]
        else:
            neg_keep = rng.choice(neg, size=n_keep, replace=False) if n_keep>0 else np.array([], int)
        keep_idx.extend(pos.tolist() + neg_keep.tolist())

    keep_mask = pd.Series(False, index=df.index)
    if keep_idx: keep_mask.loc[keep_idx] = True

    g = df.loc[keep_mask].groupby('qid')['label'].agg(n='size', pos='sum')
    valid_qids = g[(g['n']>=2) & (g['pos']>=1) & (g['pos']<g['n'])].index
    keep_mask &= df['qid'].isin(valid_qids)
    return keep_mask
# Tighten negative downsampling
hard_cols = tuple([c for c in ['hh_promo_te','promo_te','feat_promo_pos_rate'] if c in df.columns])
train_keep_mask = downsample_train_by_qid(df, train_mask, R=5, hardness_cols=hard_cols, rng_seed=42)

# Prefilter top-N candidates per query to make top-1 easier (standard two-stage retrieval→rerank)
def prefilter_topn_by_qid(frame, base_mask, N=60):
    # สร้างสกอร์เรียกคืนแบบง่าย
    s = pd.Series(0.0, index=frame.index, dtype=float)
    if 'hh_promo_te' in frame: s += 3.0*frame['hh_promo_te'].fillna(0)
    if 'promo_te'    in frame: s += 2.0*frame['promo_te'].fillna(0)
    if 'overlap_decay_h3' in frame: s += 1.0*frame['overlap_decay_h3'].fillna(0)
    if 'feat_hist_size'   in frame: s += 0.2*np.log1p(frame['feat_hist_size'].clip(lower=0))

    keep = pd.Series(False, index=frame.index)
    for qid, g in frame.loc[base_mask].groupby('qid', sort=False):
        idx = g.index
        # คัด top-N ด้วยการเรียงจากสกอร์ s ของอินเด็กซ์ในกลุ่ม (ไม่ใช้ DataFrame.sort_values(by=Series))
        top_idx = s.loc[idx].sort_values(ascending=False).head(N).index
        pos_idx = g.index[g['label'] == 1]
        keep.loc[top_idx] = True
        keep.loc[pos_idx] = True
    return keep

pref_train_mask = prefilter_topn_by_qid(df, train_mask, N=80)
pref_valid_mask = prefilter_topn_by_qid(df, valid_mask, N=80)

# Final masks used for training/validation
train_final_mask = train_keep_mask & pref_train_mask
valid_final_mask = valid_mask & pref_valid_mask

# Rebuild groups for LGBM
y_tr = df.loc[train_final_mask, 'label'].astype(int).values
y_va = df.loc[valid_final_mask, 'label'].astype(int).values
grp_tr_ds = df.loc[train_final_mask].groupby('qid', sort=False).size().tolist()
grp_va = df.loc[valid_final_mask].groupby('qid', sort=False).size().tolist()

In [47]:
train_keep_mask.value_counts()

,count
False,682321
True,6738


In [48]:
df_train_downsampled = df.loc[train_keep_mask].copy()
df_train_downsampled['label'].value_counts()

,count
label,
0,5475
1,1263


In [49]:
def make_scored_frame(model, Xv, base, valid_mask):
    vf = base.loc[valid_mask, ['qid','label']].copy()
    scores = model.predict(Xv)
    vf['score'] = scores + np.random.RandomState(42).normal(0, 1e-6, size=len(scores))
    return vf

def hitrate_at_k(df, k=10):
    top = df.sort_values(['qid','score'], ascending=[True, False]).groupby('qid').head(k)
    return (top.groupby('qid')['label'].max()).mean()

def recall_precision_at_k(df, k=10):
    got = df.sort_values(['qid','score'], ascending=[True, False]).groupby('qid').head(k)
    pos_per_q = df.groupby('qid')['label'].sum()
    hit_per_q = got.groupby('qid')['label'].sum()
    recall = (hit_per_q / pos_per_q.replace(0, np.nan)).mean()
    precision = got.groupby('qid')['label'].mean().mean()
    return float(recall), float(precision)

def mrr_at_k(df, k=10):
    def _mrr(g):
        labels = g.sort_values('score', ascending=False)['label'].to_numpy()[:k]
        for i, lab in enumerate(labels, 1):
            if lab == 1: return 1.0/i
        return 0.0
    return float(df.groupby('qid', group_keys=False).apply(_mrr).mean())

def map_at_k(df, k=10):
    def _ap(g):
        labels = g.sort_values('score', ascending=False)['label'].to_numpy()[:k]
        npos = labels.sum()
        if npos == 0: return 0.0
        c = labels.cumsum()
        precisions = (c / np.arange(1, len(labels)+1)) * labels
        return float(precisions.sum() / min(npos, k))
    return float(df.groupby('qid', group_keys=False).apply(_ap).mean())
from sklearn.metrics import ndcg_score

def ndcg_at_k_from_valid_frame(valid_frame, k=10):
    ndcgs = []
    for _, g in valid_frame.groupby('qid', sort=False):
        y_true = g['label'].to_numpy()
        if y_true.sum() == 0:
            continue
        y_pred = g['score'].to_numpy()
        ndcgs.append(ndcg_score(y_true.reshape(1,-1), y_pred.reshape(1,-1), k=k))
    return float(np.mean(ndcgs)) if ndcgs else 0.0

def report_at_ks(vf, name='set', ks=(1,3,5,10)):
    print(f"\n==== {name} ====")
    for K in ks:
        r, p = recall_precision_at_k(vf, K)
        hr = hitrate_at_k(vf, K)
        mrr = mrr_at_k(vf, K)
        mAP = map_at_k(vf, K)
        nd = ndcg_at_k_from_valid_frame(vf, k=K)
        print(f"K={K}  P={p:.4f}  R={r:.4f}  HR={hr:.4f}  MRR={mrr:.4f}  MAP={mAP:.4f}  NDCG={nd:.4f}")

In [50]:
def positive_valid_mask(df, valid_mask, qid_col='qid', label_col='label'):
    pos_per_q = df.loc[valid_mask].groupby(qid_col)[label_col].sum()
    return valid_mask & df[qid_col].isin(pos_per_q.index[pos_per_q > 0])


In [51]:
valid_pos_mask = positive_valid_mask(df, valid_mask)
grp_va_pos = df.loc[valid_pos_mask].groupby('qid', sort=False).size().tolist()

In [52]:
def train_and_eval_ranker(X_tr, y_tr, grp_tr, X_va, y_va, grp_va, base, valid_mask,
                          eval_pos_only=True, eval_pos_mask=None, eval_grp_va=None):
    if eval_pos_only:
        vm = eval_pos_mask if eval_pos_mask is not None else positive_valid_mask(base, valid_mask)
        X_va_eval = X_va.loc[vm]
        y_va_eval = base.loc[vm, 'label'].astype(int).values
        grp_va_eval = eval_grp_va if eval_grp_va is not None else base.loc[vm].groupby('qid', sort=False).size().tolist()
        eval_mask_to_pass = vm
    else:
        X_va_eval, y_va_eval, grp_va_eval = X_va, y_va, grp_va
        eval_mask_to_pass = valid_mask

    ranker = lgb.LGBMRanker(
        objective='lambdarank', metric='ndcg', boosting_type='gbdt',
        num_leaves=47, learning_rate=0.06, n_estimators=900,
        subsample=0.8, colsample_bytree=0.8, random_state=42,
        min_data_in_leaf=150, lambda_l2=1.5,
        feature_fraction=0.8, bagging_fraction=0.8, bagging_freq=1, verbose=-1,
        lambdarank_truncation_level=5
    )
    ranker.fit(
        X_tr, y_tr, group=grp_tr,
        eval_set=[(X_va_eval, y_va_eval)], eval_group=[grp_va_eval],
        eval_at=[5,10], callbacks=[lgb.early_stopping(100, first_metric_only=True)]
    )

    vf = make_scored_frame(ranker, X_va_eval, base, eval_mask_to_pass)
    res = {}
    for K in [1,3,5,10]:
        r,p = recall_precision_at_k(vf, K)
        res[f'Recall@{K}'] = r; res[f'Precision@{K}'] = p
        res[f'HitRate@{K}'] = hitrate_at_k(vf, K)
        res[f'MRR@{K}'] = mrr_at_k(vf, K); res[f'MAP@{K}'] = map_at_k(vf, K)
        nd = ndcg_at_k_from_valid_frame(vf, k=K)
        res[f'NDCG@{K}'] = nd
    return ranker, res

###Single Group Validation

In [53]:
y_tr = df.loc[train_keep_mask, 'label'].astype(int).values
y_va = df.loc[valid_mask, 'label'].astype(int).values
grp_tr_ds = df.loc[train_keep_mask].groupby('qid', sort=False).size().tolist()
grp_va = df.loc[valid_mask].groupby('qid', sort=False).size().tolist()

results_rows = []
for name, F in feature_groups.items():
    X_tr = F.loc[train_keep_mask].copy()
    X_va = F.loc[valid_final_mask].copy()
    _, res = train_and_eval_ranker(
        X_tr, y_tr, grp_tr_ds, X_va, y_va, grp_va,
        base=df, valid_mask=valid_final_mask,
        eval_pos_only=True, eval_pos_mask=valid_pos_mask, eval_grp_va=grp_va_pos
    )
    row = {'group': name}; row.update(res); results_rows.append(row)

sg = pd.DataFrame(results_rows)
if 'Precision@1' in sg and 'MRR@10' in sg:
    sg['score'] = 0.8*sg['NDCG@5'] + 0.2*sg['NDCG@10']
else:
    sg['score'] = sg.get('Precision@1', pd.Series(0, index=sg.index))
single_group_results = sg.sort_values('score', ascending=False)
single_group_results.head(10)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's ndcg@5: 0.25341	valid_0's ndcg@10: 0.351512
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	valid_0's ndcg@5: 0.272936	valid_0's ndcg@10: 0.366077
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	valid_0's ndcg@5: 0.207533	valid_0's ndcg@10: 0.304104
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's ndcg@5: 0.21217	valid_0's ndcg@10: 0.314528
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's ndcg@5: 0.248577	valid_0's ndcg@10: 0.33948
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0

,group,Recall@1,Precision@1,HitRate@1,MRR@1,MAP@1,NDCG@1,Recall@3,Precision@3,HitRate@3,...,MRR@5,MAP@5,NDCG@5,Recall@10,Precision@10,HitRate@10,MRR@10,MAP@10,NDCG@10,score
7,promo_ohe,0.095977,0.155080,0.155080,0.155080,0.155080,0.155080,0.263993,0.149733,0.374332,...,0.285383,0.278506,0.281506,0.628380,0.122995,0.737968,0.313974,0.296345,0.369285,0.299062
1,recency,0.067278,0.122995,0.122995,0.122995,0.122995,0.122995,0.263191,0.147950,0.368984,...,0.272638,0.269519,0.271933,0.666985,0.127807,0.775401,0.300840,0.286191,0.368798,0.291306
0,similarity,0.080558,0.155080,0.155080,0.155080,0.155080,0.155080,0.216934,0.133690,0.326203,...,0.269964,0.264780,0.248684,0.635625,0.121390,0.764706,0.305993,0.287892,0.352893,0.269526
5,popularity,0.092590,0.155080,0.155080,0.155080,0.155080,0.155080,0.232799,0.131907,0.336898,...,0.262210,0.256179,0.252095,0.567711,0.106952,0.711230,0.293339,0.277546,0.330719,0.267820
4,coverage,0.089598,0.149733,0.149733,0.149733,0.149733,0.149733,0.209753,0.119430,0.320856,...,0.264528,0.262433,0.246340,0.597874,0.119251,0.727273,0.295932,0.277906,0.341970,0.265466
3,immediate,0.072180,0.122995,0.122995,0.122995,0.122995,0.122995,0.215890,0.121212,0.315508,...,0.243316,0.239015,0.229327,0.567520,0.110160,0.689840,0.271433,0.249876,0.317584,0.246978
2,affinity,0.076763,0.117647,0.117647,0.117647,0.117647,0.117647,0.207627,0.114082,0.294118,...,0.218271,0.218687,0.215611,0.556061,0.104278,0.684492,0.252878,0.246322,0.305451,0.233579
6,temporal,0.066832,0.117647,0.117647,0.117647,0.117647,0.117647,0.199402,0.114082,0.304813,...,0.229323,0.224131,0.216988,0.543341,0.104278,0.673797,0.258647,0.235766,0.299706,0.233531


###Forward Groups Validation

In [54]:
order = single_group_results['group'].tolist()
X_tr_cum = pd.DataFrame(index=df.index); X_va_cum = pd.DataFrame(index=df.index)
cum_rows = []

for i, g in enumerate(order, 1):
    X_tr_cum = pd.concat([X_tr_cum, feature_groups[g]], axis=1)
    X_va_cum = pd.concat([X_va_cum, feature_groups[g]], axis=1)

    X_tr = X_tr_cum.loc[train_keep_mask].select_dtypes(include=['number','bool']).copy()
    X_va = X_va_cum.loc[valid_final_mask].select_dtypes(include=['number','bool']).copy()
    _, res = train_and_eval_ranker(
        X_tr, y_tr, grp_tr_ds, X_va, y_va, grp_va,
        base=df, valid_mask=valid_final_mask,
        eval_pos_only=True, eval_pos_mask=valid_pos_mask, eval_grp_va=grp_va_pos
    )
    row = {'step': i, 'added_group': g}; row.update(res); cum_rows.append(row)

forward_selection_results = pd.DataFrame(cum_rows)
forward_selection_results['score'] = 0.8*forward_selection_results['NDCG@5'] + 0.2*forward_selection_results['NDCG@10']
forward_selection_results.sort_values('score', ascending=False).head(10)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[8]	valid_0's ndcg@5: 0.268558	valid_0's ndcg@10: 0.37056
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[12]	valid_0's ndcg@5: 0.277845	valid_0's ndcg@10: 0.357977
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[95]	valid_0's ndcg@5: 0.280257	valid_0's ndcg@10: 0.362107
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's ndcg@5: 0.293258	valid_0's ndcg@10: 0.387723
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's ndcg@5: 0.249757	valid_0's ndcg@10: 0.357172
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	val

,step,added_group,Recall@1,Precision@1,HitRate@1,MRR@1,MAP@1,NDCG@1,Recall@3,Precision@3,...,MRR@5,MAP@5,NDCG@5,Recall@10,Precision@10,HitRate@10,MRR@10,MAP@10,NDCG@10,score
3,4,popularity,0.108989,0.165775,0.165775,0.165775,0.165775,0.165775,0.275045,0.153298,...,0.306774,0.306477,0.305416,0.680965,0.125668,0.780749,0.334683,0.317402,0.394607,0.323255
7,8,temporal,0.116603,0.160428,0.160428,0.160428,0.160428,0.160428,0.293570,0.153298,...,0.302317,0.299673,0.306924,0.605844,0.114439,0.727273,0.323854,0.312897,0.371863,0.319912
0,1,promo_ohe,0.095977,0.155080,0.155080,0.155080,0.155080,0.155080,0.263993,0.149733,...,0.285383,0.278506,0.281506,0.628380,0.122995,0.737968,0.313974,0.296345,0.369285,0.299062
2,3,similarity,0.111625,0.187166,0.187166,0.187166,0.187166,0.187166,0.258276,0.147950,...,0.303922,0.297482,0.277583,0.619519,0.116578,0.759358,0.333686,0.307871,0.363610,0.294788
1,2,recency,0.095034,0.139037,0.139037,0.139037,0.139037,0.139037,0.245671,0.140820,...,0.273529,0.275074,0.277971,0.612541,0.116578,0.743316,0.300350,0.288661,0.355302,0.293437
6,7,affinity,0.102215,0.149733,0.149733,0.149733,0.149733,0.149733,0.245238,0.128342,...,0.273440,0.275357,0.263785,0.599720,0.113904,0.732620,0.302962,0.291689,0.350549,0.281137
5,6,immediate,0.105730,0.165775,0.165775,0.165775,0.165775,0.165775,0.247543,0.133690,...,0.272906,0.263904,0.260163,0.617596,0.116043,0.737968,0.304405,0.280480,0.352887,0.278708
4,5,coverage,0.090272,0.144385,0.144385,0.144385,0.144385,0.144385,0.221696,0.131907,...,0.266756,0.265597,0.252446,0.651681,0.118717,0.780749,0.302920,0.285856,0.357079,0.273373


In [55]:
best_step = int(forward_selection_results.sort_values('score', ascending=False).iloc[0]['step'])
selected_groups = forward_selection_results.sort_values('step').iloc[:best_step]['added_group'].tolist()

selected_cols = []
for g in selected_groups:
    selected_cols.extend(feature_groups[g].columns.tolist())
# de-dup โดยคงลำดับเดิม
selected_cols = list(dict.fromkeys(selected_cols))
# คอลัมน์คีย์/อ้างอิง (เอาเท่าที่มีอยู่จริงใน df)
id_cols = [c for c in ['household_key','BASKET_ID','PROMO_KEY','day_t','qid','label'] if c in df.columns]

# สร้างและเซฟ CSV
df_selected = df[id_cols + selected_cols].copy()
df_selected.to_csv('selected_features101.csv', index=False)
print('saved:', df_selected.shape, '→ selected_features.csv')

saved: (689059, 37) → selected_features.csv


###Selected Columns Vaalidation

In [56]:
# 1) รวมคอลัมน์ผู้สมัครจากกลุ่มที่คัดแล้ว
candidate_cols = []
for g in selected_groups:
    candidate_cols.extend(feature_groups[g].columns.tolist())
candidate_cols = list(dict.fromkeys(candidate_cols))

# 2) Prefilter: ตัดคอลัมน์คงที่/เกือบคงที่ และกรองความสัมพันธ์สูง
df_num = df[candidate_cols].select_dtypes(include=['number','bool']).copy()
# คงที่
const_mask = df_num.nunique(dropna=False) <= 1
df_num = df_num.loc[:, ~const_mask]
# ความสัมพันธ์สูง (optional, เร็วพอประมาณ)
corr = df_num.corr(method='spearman').abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [c for c in upper.columns if (upper[c] > 0.95).any()]
df_num = df_num.drop(columns=to_drop, errors='ignore')

# 3) Warm-up importance (โมเดลเล็กเพื่อจัดลำดับคอลัมน์เร็ว)
X_tr_w = df_num.loc[train_keep_mask].copy()
X_va_w = df_num.loc[valid_mask].copy()
y_va = df.loc[valid_mask, 'label'].astype(int).values
grp_va = df.loc[valid_mask].groupby('qid', sort=False).size().tolist()

# โมเดลเล็ก/เร็ว
ranker_w = lgb.LGBMRanker(objective='lambdarank', metric='ndcg',
                          num_leaves=31, n_estimators=200, learning_rate=0.08,
                          feature_fraction=0.8, bagging_fraction=0.8, bagging_freq=1,
                          random_state=42, verbose=-1, lambdarank_truncation_level=5)
# cast bool → uint8
for X_ in (X_tr_w, X_va_w):
    for c in X_.columns:
        if X_[c].dtype == bool: X_[c] = X_[c].astype(np.uint8)

ranker_w.fit(X_tr_w, y_tr, group=grp_tr_ds,
             eval_set=[(X_va_w, y_va)], eval_group=[grp_va],
             eval_at=[5,10], callbacks=[lgb.early_stopping(50, first_metric_only=True)])

imp = pd.Series(ranker_w.feature_importances_, index=X_tr_w.columns).sort_values(ascending=False)
top_cols = imp.head(200).index.tolist()  # จำกัด 200 คอลัมน์แรกก่อน

# 4) Greedy forward per-column (โฟกัส Precision@1)
chosen, best_score = [], -1.0
def eval_cols(cols):
    X_tr = df.loc[train_keep_mask, cols].select_dtypes(include=['number','bool']).copy()
    X_va = df.loc[valid_final_mask,       cols].select_dtypes(include=['number','bool']).copy()
    model, res = train_and_eval_ranker(X_tr, y_tr, grp_tr_ds, X_va, y_va, grp_va, base=df, valid_mask=valid_mask)
    return model, 0.8*res['NDCG@5'] + 0.2*res['NDCG@10']

for c in top_cols:
    trial = chosen + [c]
    _, score = eval_cols(trial)
    if score > best_score + 1e-5:  # margin เล็กน้อยกัน overfit noise
        chosen, best_score = trial, score

# (optional) floating backward: ลองถอดหนึ่งคอลัมน์ออกถ้าช่วยเพิ่มคะแนน
improved = True
while improved and len(chosen) > 1:
    improved = False
    for c in list(chosen):
        trial = [x for x in chosen if x != c]
        _, score = eval_cols(trial)
        if score > best_score + 1e-5:
            chosen, best_score = trial, score
            improved = True

selected_cols = chosen
print(f"Selected {len(selected_cols)} columns; score={best_score:.6f}")

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[179]	valid_0's ndcg@5: 0.985209	valid_0's ndcg@10: 0.986776
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's ndcg@5: 0.205325	valid_0's ndcg@10: 0.30205
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[20]	valid_0's ndcg@5: 0.260636	valid_0's ndcg@10: 0.341514
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[14]	valid_0's ndcg@5: 0.302766	valid_0's ndcg@10: 0.378792
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	valid_0's ndcg@5: 0.306386	valid_0's ndcg@10: 0.392495
Evaluated only: ndcg@5
Training until validation scores don't improve for 100 rounds
Early stopping, best iterati

In [57]:
selected_cols

['feat_hist_size',
 'promo_offer_ct',
 'feat_mean_recency',
 'feat_min_recency',
 'BRAND_Private']

In [58]:
import json

# 1) คอลัมน์อ้างอิง + ฟีเจอร์ที่เลือก
id_cols = [c for c in ['household_key','BASKET_ID','PROMO_KEY','day_t','qid'] if c in df.columns]
use_cols = selected_cols  # ชุด per-column selection ของคุณ
cols_to_keep = id_cols + use_cols + ['label']

# 2) สร้างชุดเทรน/วาลิเดตสุดท้าย (ใช้ mask หลัง downsample + prefilter)
df_train = df.loc[train_final_mask, cols_to_keep].copy()
df_valid = df.loc[valid_final_mask, cols_to_keep].copy()

# 3) เรียงตาม qid เพื่อให้ group sizes ตรงกับลำดับแถว
df_train = df_train.sort_values(['qid']).reset_index(drop=True)
df_valid = df_valid.sort_values(['qid']).reset_index(drop=True)

# 4) group sizes ต่อ qid (สำหรับ LGBM/XGB Ranker)
grp_tr = df_train.groupby('qid', sort=False).size().tolist()
grp_va = df_valid.groupby('qid', sort=False).size().tolist()

# 5) บันทึกไฟล์ (parquet เร็ว/กะทัดรัดกว่า; ใช้ csv ก็ได้)
df_train.to_parquet('train_rank.parquet', index=False)
df_valid.to_parquet('valid_rank.parquet', index=False)
with open('groups.json','w') as f:
    json.dump({'grp_tr': grp_tr, 'grp_va': grp_va}, f)

# 6) เซฟรายชื่อฟีเจอร์ที่ใช้ เพื่อให้โน้ตบุ๊กเทรนโหลดได้ตรงกัน
with open('selected_cols.json','w') as f:
    json.dump({'use_cols': use_cols, 'id_cols': id_cols}, f)

# (optional) รายชื่อ qid ที่มี positive ใน valid สำหรับ pos-only evaluation ในโน้ตบุ๊กเทรน
pos_qids = df_valid.groupby('qid')['label'].sum()
pos_qids = pos_qids.index[pos_qids>0].tolist()
with open('valid_pos_qids.json','w') as f:
    json.dump({'valid_pos_qids': pos_qids}, f)
print('Saved train_rank.parquet, valid_rank.parquet, groups.json, selected_cols.json, valid_pos_qids.json')

Saved train_rank.parquet, valid_rank.parquet, groups.json, selected_cols.json, valid_pos_qids.json


In [59]:
import json, pandas as pd
train = pd.read_parquet('train_rank.parquet')
valid = pd.read_parquet('valid_rank.parquet')
with open('groups.json') as f:
    g = json.load(f)
with open('selected_cols.json') as f:
    meta = json.load(f)

use_cols = meta['use_cols']
grp_tr, grp_va = g['grp_tr'], g['grp_va']

X_tr = train[use_cols]
y_tr = train['label'].astype(int).values
X_va = valid[use_cols]
y_va = valid['label'].astype(int).values

# เทรน LGBMRanker (ปรับ eval_at ตามเป้าหมาย เช่น [5,10])
import lightgbm as lgb
ranker = lgb.LGBMRanker(objective='lambdarank', metric='ndcg',
                        num_leaves=63, learning_rate=0.05, n_estimators=4000,
                        lambdarank_truncation_level=5, random_state=42, verbose=-1)
ranker.fit(X_tr, y_tr, group=grp_tr,
           eval_set=[(X_va, y_va)], eval_group=[grp_va],
           eval_at=[5,10], callbacks=[lgb.early_stopping(300, first_metric_only=True)])

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[119]	valid_0's ndcg@5: 0.984918	valid_0's ndcg@10: 0.986404
Evaluated only: ndcg@5


LGBMRanker(lambdarank_truncation_level=5, learning_rate=0.05, metric='ndcg',
           n_estimators=4000, num_leaves=63, objective='lambdarank',
           random_state=42, verbose=-1)

In [60]:
selected_cols

['feat_hist_size',
 'promo_offer_ct',
 'feat_mean_recency',
 'feat_min_recency',
 'BRAND_Private']

In [61]:
selected_groups

['promo_ohe', 'recency', 'similarity', 'popularity']

In [62]:
import json
def save_selected_matrix(name, cols, include_id=True, split=True):
    # คอลัมน์อ้างอิงที่แนะนำให้ติดไปด้วยเพื่อเทรน ranker
    id_cols = [c for c in ['household_key','BASKET_ID','PROMO_KEY','day_t','qid','label'] if c in df.columns]
    use_cols = (id_cols + list(cols)) if include_id else list(cols)

    # ทั้งชุด (เหมือน features_all แต่คอลัมน์ถูกกรอง)
    df_all = df[use_cols].copy()
    df_all.to_csv(f'features_{name}_all.csv', index=False)

    if split:
        # แยกตาม mask ที่เตรียมไว้ (หลัง downsample + prefilter)
        df_tr = df.loc[train_final_mask, use_cols].copy()
        df_va = df.loc[valid_final_mask, use_cols].copy()
        df_tr.to_csv(f'features_{name}_train.csv', index=False)
        df_va.to_csv(f'features_{name}_valid.csv', index=False)

        # เก็บ group sizes เผื่อเทรน LGBMRanker
        grp_tr = df_tr.groupby('qid', sort=False).size().tolist() if 'qid' in df_tr else []
        grp_va = df_va.groupby('qid', sort=False).size().tolist() if 'qid' in df_va else []
        with open(f'groups_{name}.json','w') as f:
            json.dump({'grp_tr': grp_tr, 'grp_va': grp_va}, f)

    print(f"Saved CSVs for '{name}'")

# ตัวอย่างหลายเวอร์ชัน
save_selected_matrix('forward_best', selected_cols)        # union ของกลุ่มถึงสเต็ปที่ดีที่สุด

Saved CSVs for 'forward_best'
